In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

ROBOFLOW_API = userdata.get('ROBOFLOW_API')
rf = Roboflow(api_key=ROBOFLOW_API)

project = rf.workspace("mb-fbwt7").project("keypoints-ppgvd")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

In [ ]:
%cd {HOME}

!yolo task=pose mode=train model=yolov8x-pose.pt save=True pretrained=/content/best.pt data={dataset.location}/data.yaml batch=48 epochs=500 imgsz=640 mosaic=0.0 plots=True

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/pose/train/val_batch0_pred.jpg', width=600)

In [ ]:
%cd {HOME}

!yolo task=pose mode=val model=/content/best.pt data={dataset.location}/data.yaml

In [ ]:
project.version(dataset.version).deploy(model_type="yolov11n-pose", model_path=f"{HOME}/runs/pose/train/")